# Comparing various values from ISIS and USGSCSM cameras for a Messenger MDIS NAC PDS3 image

In [1]:
import os
os.environ['ISISROOT'] = '/usr/local/Caskroom/miniconda/base/envs/isis-prod'
os.environ['ISISDATA'] = '/Volumes/usgs-data/isis-data'
import urllib.request

import ale
from ale.drivers.mess_drivers import MessengerMdisPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import os
import kalasiris as isis
import pvl
import numpy as np
import knoten
import csmapi
from knoten import csm

ale.spice_root = '/Volumes/usgs-data/spice-data'

In [2]:
# printing config displays the yaml formatted string
# print(ale.config) Gone?

# config object is a dictionary so it has the same access patterns 
# print('MDIS spice directory:', ale.config['mdis'])

# updating config for new MDIS path in this notebook 
# Note: this will not change the path in `.ale/config.yml`. This change only lives in the notebook.
# ale.config['mdis'] = ''

In [3]:
# https://pdsimage2.wr.usgs.gov/Messenger/MSGRMDS_1001/DATA/2015_114/
# https://d3fhgbbgskqro0.cloudfront.net/MSGRMDS_1001/DATA/2015_114/EN1072174528M.IMG

imgurl = 'https://d3fhgbbgskqro0.cloudfront.net/MSGRMDS_1001/DATA/2015_114/EN1072174528M.IMG'
image_dir = 'data'
EN1072174528M = os.path.join(image_dir, 'EN1072174528M.IMG')
downloader = urllib.request.URLopener()
if not os.path.isfile(EN1072174528M):
    downloader.retrieve(imgurl, EN1072174528M)

In [6]:
# change to desired PDS3 image path 
fileName = EN1072174528M

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MessengerMdisPds3NaifSpiceDriver(fileName) as driver:
    # Get rotation from target_frame to j2000
    j2000 = driver.frame_chain
    target_frame = j2000.find_child_frame(driver.target_frame_id)
    rotation = target_frame.rotation_to(j2000)
    
    # Apply rotation to sensor position and velocity
    j2000RotationPos = rotation._rots.apply(driver.sensor_position[0])
    j2000RotationVel = rotation._rots.apply(driver.sensor_position[1])

    # pass driver instance into formatter function
    usgscsmString = to_usgscsm(driver)

AttributeError: 'FrameChain' object has no attribute 'find_child_frame'

In [4]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)

In [5]:
# Constructs a camera model using usgscsm
model="USGS_ASTRO_FRAME_SENSOR_MODEL"  # Make sure this matches your camera model
plugin = csmapi.Plugin.getList()[0]
isd = csmapi.Isd(fileName)
warns = csmapi.WarningList()
if plugin.canModelBeConstructedFromISD(isd, model, warns):
    print("CONSTRUCTED CAMERA")
    camera = plugin.constructModelFromISD(isd, model)
else:
    print("CAN'T CONSTRUCT CAMERA")
    for item in warns:
        print(item.getMessage())

CONSTRUCTED CAMERA


In [ ]:
# Ingest image and spiceinit it
cube = os.path.splitext(fileName)[0] + '.cub'
isis.mdis2isis(from_=fileName, to=cube)
isis.spiceinit(from_=cube, shape='ellipsoid')

In [7]:
# Grab campt output on spiceinit'd cube and load it as a pvl
output = isis.campt(from_=cube)
pvl_output = pvl.loads(output)

In [8]:
# Grab body fixed coordinates from campt pvl output
campt_bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
campt_bodyfixed = np.asarray(campt_bodyfixed.value) * 1000

# Grab body fixed coordinates from csm
ale_bodyfixed = csm.generate_ground_point(0, (256 - .5, 256 - .5), camera)
ale_bodyfixed = np.array([ale_bodyfixed.x, ale_bodyfixed.y, ale_bodyfixed.z])

# Compare the two body fixed coordinates
ale_bodyfixed - campt_bodyfixed

array([ 124.214954  , -139.13882436,  -79.08708214])

In [9]:
# Grab sensor position from isd
ale_position = usgscsm_dict['sensor_position']['positions']
ale_position = np.asarray(ale_position)

# Grab spacecraft position from campt pvl output
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000

# Compare the two positions
ale_position - campt_position

array([[ -66.51548142, -182.55737144,  248.80339983]])

In [10]:
# Grab InstrumentPosition table from the isis cube using tabledump
instrument_pos_table = str(isis.tabledump(from_=cube, name='InstrumentPosition'))
parsed_string = instrument_pos_table.split(',')

In [11]:
# Grab sensor position from the table dump output
isis_j2000_pos = np.asarray([float(parsed_string[6][4:]), float(parsed_string[7]), float(parsed_string[8])]) * 1000

# Grab ALE's sensor position
ale_j2000_pos = np.asarray(j2000RotationPos)

# Compare the two sensor positions that are in the j2000 reference frame
ale_j2000_pos - isis_j2000_pos

array([[ 215.13810917, -126.54603034,  193.27785975]])

In [12]:
# Grab velocities from the table dump output
isis_j2000_vel = np.asarray([float(parsed_string[9]), float(parsed_string[10]), float(parsed_string[11])]) * 1000

# Grab ALE's velocities
ale_j2000_vel = np.asarray(j2000RotationVel)

# Compare the two velocity lists that are in the j2000 reference frame
ale_j2000_vel - isis_j2000_vel

array([[-1.00183497, -1.69444679, -0.87865924]])

In [13]:
# Grab spacecraft position and body fixed look vector from csm
locus = camera.imageToRemoteImagingLocus(csmapi.ImageCoord(256 - .5, 256 - .5))
csm_bodyfixedLV = np.asarray([locus.direction.x, locus.direction.y, locus.direction.z])
csm_position = np.asarray([locus.point.x, locus.point.y, locus.point.z])

# Grab spacecraft position and body fixed look vector from campt pvl output
campt_bodyfixedLV = np.asarray(pvl_output['GroundPoint']['LookDirectionBodyFixed'])
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000

# Compute the differences
print(csm_bodyfixedLV - campt_bodyfixedLV)
print(csm_position - campt_position)

[ 3.20945290e-05 -4.88871593e-03 -4.34349143e-03]
[ -66.51548142 -182.55737144  248.80339983]
